In [ ]:
from __future__ import print_function
import os
import neat
import visualize

import numpy as np
import pygame
import tkinter as tk
from tkinter import messagebox

class options:
    
    # Theese are tweakable. Adjust to fit your needs.
    
    ScreenWidth = 1000
    ArenaSize = 50                      # In number of squares
    FPSLimit = 60
    SnakeSpeed = 8                    # I recommend integers from 1 to 10 and I dare you to score 10 points on 10.
    TurnsTillStarve = 150
    AmountOfSnacks = 1
    NrOfNNAttempts = 1
    
    SpaceForText = 200 * ScreenWidth // 1000
    delay = int(200 // SnakeSpeed)
    UnitWidth = ScreenWidth // ArenaSize

class colors:
        
    gold = 255, 215, 0
    silver = 192, 192, 192
    black = 0, 0, 0
    green = 0, 128, 0
    red = 255, 0, 0
    cyan = 0, 255, 255
    white = 255, 255, 255
    
class SquareOrCircle:
    
    def __init__(self, coords, color):
        
        self.coords = coords
        self.color = color
        self.vx = 0
        self.vy = -1
        self.UW = options.UnitWidth
        self.SFT = options.SpaceForText 

    def move(self, vx, vy):
        
        self.vx = vx
        self.vy = vy
        self.coords = (self.coords[0] + self.vx, self.coords[1] + self.vy)

    def DrawSquare(self, surface, eyes=False):
            
        pygame.draw.rect(surface, self.color, (self.coords[0]*self.UW+1+self.SFT, self.coords[1]*self.UW+1, self.UW-1, self.UW-1))
        
        if eyes:
            centre = self.UW // 2
            radius = 3 * options.ScreenWidth // 1000
            CircleMiddle = (self.coords[0]*self.UW+centre-radius+self.SFT, self.coords[1]*self.UW+8)
            CircleMiddle2 = (self.coords[0]*self.UW+self.UW-radius*2+self.SFT, self.coords[1]*self.UW+8)
            pygame.draw.circle(surface, colors.black, CircleMiddle, radius)
            pygame.draw.circle(surface, colors.black, CircleMiddle2, radius)
            
    def DrawCircle(self, surface):
        
        CircleMiddle = (self.coords[0]*self.UW+self.UW//2+self.SFT, self.coords[1]*self.UW+self.UW//2)
        pygame.draw.circle(surface, self.color, CircleMiddle, self.UW//2)

class wall:
        
    def __init__(self):
        
        self.walls = []
        self.R = options.ArenaSize
        
    def build(self):
        
        B = colors.black
    
        for j in range (self.R):
            self.walls.append(SquareOrCircle((j, 0), color=B))
            self.walls.append(SquareOrCircle((j, self.R - 1), color=B))
        
            if j != 0 and j != self.R - 1:
                self.walls.append(SquareOrCircle((0, j), color=B))
                self.walls.append(SquareOrCircle((self.R - 1, j), color=B))
                
            if 1.5 * self.R // 5 <= j < 3.5 * self.R // 5:
                self.walls.append(SquareOrCircle((self.R // 3, j), color=B))
                self.walls.append(SquareOrCircle((2 * self.R // 3, j), color=B))
                
    
    def draw(self, surface):
        
        for x in self.walls:
            x.DrawSquare(surface)
    
class food:
    
    def __init__(self, wall, snake1, snake2, TP):
        
        self.snacks = []
        
        for i in range(options.AmountOfSnacks):
            self.AddSnackInRandomPosition(wall, snake1, snake2, False)
            self.AddSnackInRandomPosition(wall, snake1, snake2)
                
        if TP == True:
            for i in range(options.AmountOfSnacks):
                self.AddSnackInRandomPosition(wall, snake1, snake2, False)
                self.AddSnackInRandomPosition(wall, snake1, snake2)
        
    def AddSnack(self, coords, tasty=True):
        
        if tasty == True:
            self.snacks.append(SquareOrCircle(coords, color=colors.red))
        else:
            self.snacks.append(SquareOrCircle(coords, color=colors.cyan))

    
    def AddSnackInRandomPosition(self, wall, snake1, snake2, tasty=True):
        
        R = options.ArenaSize
        snakes = snake1.body + snake2.body

        while True:
            x = np.random.randint(1, R)
            y = np.random.randint(1, R)
            
            condition1 = len(list(filter(lambda z:z.coords == (x, y), snakes))) > 0
            condition2 = len(list(filter(lambda z:z.coords == (x, y), wall))) > 0
            condition3 = len(list(filter(lambda z:z.coords == (x, y), self.snacks))) > 0
            
            if condition1 or condition2 or condition3:
                continue
                
            else:
                if tasty == True:
                    self.snacks.append(SquareOrCircle((x, y), color=colors.red))
                    break

                else:
                    self.snacks.append(SquareOrCircle((x, y), color=colors.cyan))
                    break
    
    def draw(self, surface):
        
        for x in self.snacks:
            x.DrawCircle(surface)
    
class snake:
    
    def __init__(self, coords, color):
        
        self.color = color
        self.start = coords
        self.body = []
        self.turns = {}
        self.head = SquareOrCircle(self.start, self.color)
        self.body.append(self.head)
        self.vx = 0
        self.vy = -1
        self.RIP = False
        self.dead = False
        self.score = 0
        self.starve = options.TurnsTillStarve

    def ArrowInput(self):

        self.keys = pygame.key.get_pressed()

        for key in self.keys:
            if self.keys[pygame.K_LEFT] and self.vx != 1 and self.vx != -1:
                self.vx = -1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_RIGHT] and self.vx != 1 and self.vx != -1:
                self.vx = 1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_UP] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = -1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_DOWN] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = 1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]
                
    def WASDInput(self):
        
        self.keys = pygame.key.get_pressed()

        for key in self.keys:
            if self.keys[pygame.K_a] and self.vx != 1 and self.vx != -1:
                self.vx = -1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_d] and self.vx != 1 and self.vx != -1:
                self.vx = 1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_w] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = -1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_s] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = 1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

    def NN_move(self, output):

        if output[1] > output[0] and output[1] > output[2]:
            if self.vx == 1:
                self.vx = 0
                self.vy = 1
            elif self.vx == -1:
                self.vx = 0
                self.vy = -1
            elif self.vy == 1:
                self.vx = -1
                self.vy = 0
            elif self.vy == -1:
                self.vx = 1
                self.vy = 0
            self.turns[self.head.coords[:]] = [self.vx, self.vy]

        elif output[2] > output[0] and output[2] > output[1]:
            if self.vx == 1:
                self.vx = 0
                self.vy = -1
            elif self.vx == -1:
                self.vx = 0
                self.vy = 1
            elif self.vy == 1:
                self.vx = 1
                self.vy = 0
            elif self.vy == -1:
                self.vx = -1
                self.vy = 0
            self.turns[self.head.coords[:]] = [self.vx, self.vy]
                
    def move(self, output, arrows=True, bot=True):
        
        if bot == False:
            if arrows == True:
                self.ArrowInput()
            else:
                self.WASDInput()
        else:
            self.NN_move(output)
            
        for i, c in enumerate(self.body):
            p = c.coords[:]

            if p in self.turns:
                turn = self.turns[p]
                c.move(turn[0],turn[1])

                if i == len(self.body)-1:
                    self.turns.pop(p)
            else:
                c.move(c.vx, c.vy)

        self.starve -= 1 

    def AddSquare(self):
        
        tail = self.body[-1]
        dx, dy = tail.vx, tail.vy

        if dx == 1 and dy == 0:
            self.body.append(SquareOrCircle((tail.coords[0]-1,tail.coords[1]), self.color))
        elif dx == -1 and dy == 0:
            self.body.append(SquareOrCircle((tail.coords[0]+1,tail.coords[1]), self.color))
        elif dx == 0 and dy == 1:
            self.body.append(SquareOrCircle((tail.coords[0],tail.coords[1]-1), self.color))
        elif dx == 0 and dy == -1:
            self.body.append(SquareOrCircle((tail.coords[0],tail.coords[1]+1), self.color))

        self.body[-1].vx = dx
        self.body[-1].vy = dy
        
    def RemoveSquare(self):
        
        if len(self.body) > 1:
            del self.body[-1]
        else:
            self.RIP = True

    def reset(self):
        
        self.head = SquareOrCircle(self.start, self.color)
        self.body = []
        self.body.append(self.head)
        self.turns = {}
        self.vx = 0
        self.vy = -1
        self.starve = options.TurnsTillStarve
        self.dead = False

    def draw(self, surface):
        
        for i, j in enumerate(self.body):
            if i == 0:
                j.DrawSquare(surface, eyes=True)
            else:
                j.DrawSquare(surface)

class game:
    
    def __init__(self):
        
        self.quit = False
        self.R = options.ArenaSize
        self.attempts = options.NrOfNNAttempts
        self.AmountOfSnacks = options.AmountOfSnacks
        
        self.dead = False
        self.wall = wall()
        self.wall.build()
        self.snake1 = snake((7*self.R//8, 7*self.R//8), colors.gold)
        self.snake2 = snake((self.R//8, 7*self.R//8), colors.silver)
        
    def play(self, net, TwoPlayers=False, render=True, sound=True):
        
        if TwoPlayers == True:
            self.MaxNumberOfSnacks = self.AmountOfSnacks * 4
            
        else:
            self.MaxNumberOfSnacks = self.AmountOfSnacks * 2
            self.snake2.body = []
            
        self.food = food(self.wall.walls, self.snake1, self.snake2, TwoPlayers)
        TimeSurvived = 0 
        
        if render == True:
            pygame.init() 
            ScreenSize = options.ScreenWidth + options.SpaceForText, options.ScreenWidth
            window = pygame.display.set_mode(ScreenSize)
            pygame.display.set_caption('Eat or be eaten!')
            clock = pygame.time.Clock()
            if sound == True:
                pygame.mixer.init()
                pygame.mixer.music.load('E:/Creativity/Python/Snak/GB.mp3') # Insert your favorite track here with path to it 
                pygame.mixer.music.play(-1)
                
        while True:  
            if render == True:
                pygame.time.delay(options.delay)
                clock.tick(options.FPSLimit)
                self.KeyboardInput()
            
            if self.snake1.dead == False:
                TimeSurvived += 1
                i = self.NN_input(TwoPlayers)
                output = net.activate(i)
                #print(output)
                self.snake1.move(output)
                self.CheckForPreyAndEat(self.snake1)
                self.collision(self.snake1, 1, render, TwoPlayers)
            
            if TwoPlayers == True:
                if self.snake2.dead == False:
                    if self.snake1.dead == False:
                        self.CollisionHeadOn(render, TwoPlayers)
                    self.snake2.move(0, True, False)
                    self.CheckForPreyAndEat(self.snake2)
                    self.collision(self.snake2, 2, render, TwoPlayers)
                    if self.snake1.dead == False:
                        self.CollisionHeadOn(render, TwoPlayers)
                        
                if self.snake1.dead == True and self.snake2.dead == True:
                    if render == True:
                        if self.dead == False:
                            self.message = 'Suspicious... both players suicided synchronically. \nPlayer 1 score: ' + str(max(0, len(self.snake1.body)-1)) + \
                            '\nPlayer 2 score: ' + str(max(0, len(self.snake2.body)-1)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                        self.note = 'R.I.P. poor snakes'
                        self.message_box(self.note, self.message)
                    self.reset(TwoPlayers)
                        
                if self.snake1.dead == True or self.snake2.dead == True:
                    self.dead = True
            else:            
                if self.snake1.dead == True or self.snake2.dead == True:
                    if render == True:
                        #self.note = 'R.I.P. poor snakes'
                        #self.message_box(self.note, self.message)
                        #self.reset(TwoPlayers)
                        self.quit = True
                    else:
                        self.quit = True
                                
            if self.quit == True:
                break 
                    
            if render == True:
                self.RedrawWindow(window, TwoPlayers)
                
        if render == True:
            if sound == True:
                pygame.mixer.music.stop
            pygame.quit()
            
        score = self.snake1.score + TimeSurvived / 150
        return score
    
    def KeyboardInput(self):
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        if pygame.key.get_pressed()[pygame.K_ESCAPE]:
            self.quit = True
            
    def RedrawWindow(self, window, TwoPlayers=False):
        
        window.fill(colors.green)
        self.wall.draw(window)
        
        self.RenderText('2Psnake', 1, window, 40*options.ScreenWidth//1000, False)
        self.RenderText('Snake speed '+ str(options.SnakeSpeed), 49, window)
        self.RenderText('Player 1 score: ' + str(self.snake1.score), 5, window)
        self.RenderText('Food reserve: ' + str(self.snake1.starve), 6, window)
        
        self.food.draw(window)
        self.snake1.draw(window)
        
        if TwoPlayers == True:
            self.snake2.draw(window)
            self.RenderText('Player 2 score: ' + str(self.snake2.score), 8, window)
            self.RenderText('Food reserve: ' + str(self.snake2.starve), 9, window)
        
        pygame.display.update()

    def RenderText(self, message, row, window, ShriftSize=20*options.ScreenWidth//1000, topleft=True):
            
        font = pygame.font.Font('freesansbold.ttf', ShriftSize)
        text = font.render(message, True, colors.white, colors.green)
        TextRect = text.get_rect()
        
        if topleft == True:
            TextRect.topleft = (0, row * options.UnitWidth)
        else:
            TextRect.center = (options.SpaceForText//2, (2*row+1)*options.UnitWidth//2)
            
        window.blit(text, TextRect)
    
    def message_box(self, subject, content):
        
        root = tk.Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        messagebox.showinfo(subject, content)
        try:
            root.destroy()
        except:
            pass
        
    def CheckForPreyAndEat(self, snake):
                
        for j in self.food.snacks:
            if snake.body[0].coords == j.coords:
                
                snake.starve += 75
                NumberOfTastySnacks = 0
                NumberOfBarelyEdibleSnacks = 0

                for i in self.food.snacks:
                    if i.color == colors.red:
                        NumberOfTastySnacks += 1
                    else:
                        NumberOfBarelyEdibleSnacks += 1
                
                if j.color == colors.red:
                    snake.AddSquare()
                    if NumberOfTastySnacks <= self.MaxNumberOfSnacks // 2:
                        self.food.AddSnackInRandomPosition(self.wall.walls, self.snake1, self.snake2)
                else:
                    snake.RemoveSquare()
                    if NumberOfBarelyEdibleSnacks <= self.MaxNumberOfSnacks // 2:
                        self.food.AddSnackInRandomPosition(self.wall.walls, self.snake1, self.snake2, False)
                        
                del self.food.snacks[self.food.snacks.index(j)]
            
    def collision(self, s, x, render, TwoPlayers):
        
        s.score = len(s.body) - 1
        
        if s.body[0].coords in list(map(lambda z:z.coords, self.wall.walls)):
            if TwoPlayers == True:
                if render == True:
                    
                    if x == 1:
                        if self.snake2.dead == False:
                            self.message = 'Tsk tsk tsk... Player 1 should mind the suroundings more. \nPlayer 1 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 1 outlived Player 2 but... he should still mind the suroundings more. \nPlayer 1 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'
                            
                    else:
                        if self.snake1.dead == False:
                            self.message = 'Tsk tsk tsk... Player 2 should mind the suroundings more. \nPlayer 2 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 2 outlived Player 1 but... he should still mind the suroundings more. \nPlayer 2 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'

                self.ServeTheSnake(s)
                
            else:
                if render == True:
                    self.message = 'Tsk tsk tsk... Player ' + str(x) + ' should mind the suroundings more. \nScore: ' + str(s.score) + \
                     '\n\nPress OK to play again. You can always quit with ESCAPE.'
                    
            s.dead = True       

        elif s.RIP == True:
            if TwoPlayers == True:
                if render == True:
                    if x == 1:
                        
                        if self.snake2.dead == False:
                            self.message = 'Player 1 should\'ve taken the red pill. \nPlayer 1 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 1 outlived Player 2 but... he should\'ve still taken the red pill. \nPlayer 1 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'
                            
                    else:
                        if self.snake1.dead == False:
                            self.message = 'Player 2 should\'ve taken the red pill. \nPlayer 2 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 2 outlived Player 1 but... he should\'ve still taken the red pill. \nPlayer 2 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'

                
                self.ServeTheSnake(s)
                
            else:
                if render == True:
                    self.message = 'Player ' + str(x) + ' should\'ve taken the red pill. \nScore: ' + str(len(s.body)-1) + \
                     '\n\nPress OK to play again. You can always quit with ESCAPE.'
                    
            s.dead = True
            s.RIP = False

        elif s.starve == 0:
            if TwoPlayers == True:
                if render == True:
                    if x == 1:
                        
                        if self.snake2.dead == False:
                            self.message = 'Player 1 starved to death. \nPlayer 1 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 1 outlived Player 2 but... he still starved to death. \nPlayer 1 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'
                            
                    else:
                        if self.snake1.dead == False:
                            self.message = 'Tsk tsk tsk... Player 2 starved to death. \nPlayer 2 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 2 outlived Player 1 but... he still starved to death. \nPlayer 2 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'
                            
                self.ServeTheSnake(s)
                
            else:
                if render == True:
                    self.message = 'Player ' + str(x) + ' starved to death. \nScore: ' + str(s.score) + \
                     '\n\nPress OK to play again. You can always quit with ESCAPE.'
                    
            s.dead = True
              
        elif len(self.snake1.body) > 1:
            if s.body[0].coords in list(map(lambda z:z.coords, self.snake1.body[1:])):
                if TwoPlayers == True:
                    if render == True:

                        if x == 1:
                            if self.snake2.dead == False:
                                self.message = 'Ouroboros! Player 1 bit his tail. \nPlayer 1 score: ' + \
                                str(s.score) + '\n\n'
                            else:
                                self.message += 'Player 1 outlived Player 2 yet... he still follows the teachings of Ouroboros. \nPlayer 1 score: ' + \
                                str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'

                        elif x == 2:
                            self.message = 'Player 2 was a snack to Player 1. \nPlayer 2 score: ' + \
                            str(s.score) + '\n\n'

                    self.ServeTheSnake(s)
                    
                else:
                    if render == True:
                        self.message = 'Ouroboros! Player ' + str(x) + ' bit his tail. \nScore: ' + str(s.score) + \
                         '\n\nPress OK to play again. You can always quit with ESCAPE.'
                        
                s.dead = True
                        
        elif len(self.snake2.body) > 1:
            if s.body[0].coords in list(map(lambda z:z.coords, self.snake2.body[1:])):
                if render == True:

                    if x == 1:
                        self.message = 'Player 1 was a snack to Player 2. \nPlayer 1 score: ' + \
                        str(s.score) + '\n\n'

                    elif x == 2:
                        if self.snake1.dead == False:
                            self.message = 'Ouroboros! Player 2 bit his tail. \nPlayer 2 score: ' + \
                            str(s.score) + '\n\n'
                        else:
                            self.message += 'Player 2 outlived Player 1 yet... he still follows the teachings of Ouroboros. \nPlayer 2 score: ' + \
                            str(s.score) +'\n\nPress OK to play again. You can always quit with ESCAPE.'

                self.ServeTheSnake(s)
                s.dead = True
    
    def CollisionHeadOn(self, render, TwoPlayers):
        if len(self.snake1.body) > 0 and len(self.snake2.body) > 0:
            if self.snake1.body[0].coords == self.snake2.body[0].coords:
                if render == True:
                    self.message = 'Head-on! That had to hurt... \n\nPlayer 1 score: ' + str(len(self.snake1.body)-1) + \
                    '\nPlayer 2 score: ' + str(len(self.snake2.body)-1) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                    self.note ='R.I.P. poor snakes'
                    self.message_box(self.note, self.message)
                self.reset(TwoPlayers)
    
    def ServeTheSnake(self, snake):
        
        for a, b in enumerate(snake.body):
            if a % 2 == 1:
                self.food.AddSnack(b.coords)
        snake.body = []
    
    def reset(self, TwoPlayers):
        
        self.snake1.reset()
        self.snake2.reset()
        self.food.snacks = []
        for i in range(self.AmountOfSnacks):
            self.food.AddSnackInRandomPosition(self.wall.walls, self.snake1, self.snake2, False)
            self.food.AddSnackInRandomPosition(self.wall.walls, self.snake1, self.snake2)
        if TwoPlayers == True:
            for i in range(self.AmountOfSnacks):
                self.food.AddSnackInRandomPosition(self.wall.walls, self.snake1, self.snake2, False)
                self.food.AddSnackInRandomPosition(self.wall.walls, self.snake1, self.snake2)
        self.dead = False
        
    def NN_input(self, TwoPlayers):
        
        snake = self.snake1.body[0]
        coords = [snake.coords] * 7
        typ = [1] * 7
        distance = [0] * 7
        if self.snake1.vx == 1:
            move = [(1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        elif self.snake1.vx == -1:
            move = [(-1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
        elif self.snake1.vy == 1:
            move = [(0, 1), (-1, 0), (1, 0), (-1, 1), (1, 1), (-1, -1), (1, -1)]
        elif self.snake1.vy == -1:
            move = [(0, -1), (1, 0), (-1, 0), (1, -1), (-1, -1), (1, 1), (-1, 1)]
        
        while True:
            for i in range(7):
                if typ[i] != 1:
                    continue
                    
                coords[i] = (coords[i][0] + move[i][0], coords[i][1] + move[i][1])
                
                if coords[i] in list(map(lambda z:z.coords, self.wall.walls)):
                    distance[i] = (abs(snake.coords[0] - coords[i][0]) + \
                                   abs(snake.coords[1] - coords[i][1])) / 10
                    typ[i] = -3
                    
                elif coords[i] in list(map(lambda z:z.coords, self.snake1.body)):
                    distance[i] = (abs(snake.coords[0] - coords[i][0]) + \
                                   abs(snake.coords[1] - coords[i][1])) / 10
                    typ[i] = -5
                
                elif TwoPlayers == True:
                    if coords[i] in list(map(lambda z:z.coords, self.snake2.body)):
                        distance[i] = (abs(snake.coords[0] - coords[i][0]) + \
                                       abs(snake.coords[1] - coords[i][1])) / 10
                        typ[i] = -5
                
                elif coords[i] in list(map(lambda z:z.coords, self.food.snacks)):
                    distance[i] = (abs(snake.coords[0] - coords[i][0]) + \
                                   abs(snake.coords[1] - coords[i][1])) / 10
                    for j in self.food.snacks:
                        if coords[i] == j.coords:
                            if j.color == colors.red:
                                typ[i] = 5
                            else:
                                typ[i] = 0
                    
            if 1 not in typ:
                break
        
        inp = (distance[0], distance[1], distance[2], distance[3], distance[4], distance[5],
               distance[6], typ[0], typ[1], typ[2], typ[3], typ[4], typ[5], typ[6],
               self.snake1.starve / 150, self.snake1.score / 10)
        #print(inp)
        return inp
            
    def eval_genomes(self, genomes, config):

        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)
            score = []
            for i in range(self.attempts):
                score.append(self.play(net, TwoPlayers=False, render=False, sound=False))
                self.reset(False)
                self.quit = False
            genome.fitness = np.mean(score)
        
    def run(self, config_file):
        
        # Load configuration.
        config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                             neat.DefaultSpeciesSet, neat.DefaultStagnation,
                             config_file)

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)
        #p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-99')

        # Add a stdout reporter to show progress in the terminal.
        p.add_reporter(neat.StdOutReporter(True))
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        p.add_reporter(neat.Checkpointer(10, None))

        # Run for up to 300 generations.
        winner = p.run(self.eval_genomes, 150)
        # Display the winning genome.
        print('\nBest genome:\n{!s}'.format(winner))

        #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
        #visualize.draw_net(config, winner, True, node_names=node_names)
        visualize.plot_stats(stats, ylog=False, view=True)
        visualize.plot_species(stats, view=True)
        
        return winner
        
    def learn(self):
        
        if __name__ == '__main__':
            # Determine path to configuration file. This path manipulation is
            # here so that the script will run successfully regardless of the
            # current working directory.
            # os.path.dirname(__file__) => globals()['_dh'][0] to work in Jupyter notebook
            local_dir = globals()['_dh'][0]
            config_path = os.path.join(local_dir, 'config_snak.txt')
            winner = self.run(config_path)
            return winner

g = game()
winner = g.learn()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 

Population's average fitness: 0.61170 stdev: 0.44544
Best fitness: 2.50000 - size: (3, 24) - species 1 - id 133
Average adjusted fitness: 0.232
Mean genetic distance 1.556, standard deviation 0.248
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   200      2.5    0.232     0
Total extinctions: 0
Generation time: 43.401 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.73580 stdev: 0.48509
Best fitness: 3.03333 - size: (4, 24) - species 1 - id 396
Average adjusted fitness: 0.232
Mean genetic distance 1.633, standard deviation 0.231
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   200      3.0    0.232     0
Total extinctions: 0
Generation time: 53.110 sec (48.256 ave

Population's average fitness: 0.85807 stdev: 0.79315
Best fitness: 5.12667 - size: (3, 13) - species 1 - id 3683
Average adjusted fitness: 0.161
Mean genetic distance 1.731, standard deviation 0.368
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   18   200      5.1    0.161     7
Total extinctions: 0
Generation time: 40.557 sec (36.073 average)

 ****** Running generation 19 ****** 

Population's average fitness: 0.91057 stdev: 0.94556
Best fitness: 8.50000 - size: (3, 15) - species 1 - id 3864
Average adjusted fitness: 0.104
Mean genetic distance 1.768, standard deviation 0.343
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   19   200      8.5    0.104     0
Total extinctions: 0
Generation time: 42.555 sec (37.140 average)
Saving checkpoint to neat-checkpoint-19

 ****** Running generation 20 ****** 

Population's avera

Population's average fitness: 1.14113 stdev: 1.49497
Best fitness: 15.50000 - size: (3, 15) - species 2 - id 7031
Average adjusted fitness: 0.068
Mean genetic distance 1.979, standard deviation 0.482
Population of 200 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   35    70      2.9    0.047    13
     2   10   130     15.5    0.089     0
Total extinctions: 0
Generation time: 56.173 sec (45.908 average)

 ****** Running generation 36 ****** 

Population's average fitness: 1.02357 stdev: 0.97972
Best fitness: 6.80000 - size: (3, 13) - species 2 - id 7284
Average adjusted fitness: 0.137
Mean genetic distance 2.005, standard deviation 0.473
Population of 200 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   36    76      2.5    0.112    14
     2   11   118      6.8    0.162     1
     3    0     6       --       --     0
Total extinctions: 0
Generation time:

In [ ]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config_snak.txt')
net = neat.nn.FeedForwardNetwork.create(winner, config)
g.reset(False)
g.quit = False
g.play(net, TwoPlayers=False)